In [ ]:
import yfinance as yf
import numpy as np

ticker = 'TSLA'

start_date = "2015-01-01"
df = yf.download(
    ticker, 
    interval='1d',
    # start=start_date,
    period='10y'
)
df.columns = df.columns.get_level_values(0)
df = df[['Close']] # keep only close prices
df['return'] = df['Close'].pct_change()
df['state'] = np.where(df['return'] >= 0, 0, 1)

In [ ]:
df.to_csv('data.csv')

In [28]:
import numpy as np
import pandas as pd

df = pd.read_csv('data.csv', index_col = 'Date')

In [29]:
def getReturnVolatility(df, state=0, period=50): 
    if state not in df['state'].unique():
        print('Invalid state')
        return -1
    local_df = df.copy()
    local_df['segment'] = pd.qcut(range(len(local_df)), q=period, labels=False)

    volatility_per_segment = local_df[local_df['state'] == state].groupby('segment')['return'].var()
    weight = pd.Series([2**(-(period-i)) for i in range(period)])
    return np.sqrt(volatility_per_segment.dot(weight) * 252) # convert to annual vol

In [30]:
vols = [ getReturnVolatility(df, i, 100) for i in range(2) ]
vols

[np.float64(0.14669682762693306), np.float64(0.1983869414748875)]

### Phase 2: building matrices

In [31]:
import numpy as np
import pandas as pd
from functools import cache

In [32]:
# generating matrix A
n = .5
A = np.matrix([[-n, n], [n, -n]])

T = 1 # years
N = 1000
delta_t = T/N
def get_markov_transition_prob(delta_t, A, max_iter=50):
    res = np.eye(A.shape[0])
    prev = np.eye(A.shape[0])
    for i in range(max_iter):
        prev = prev @ A * delta_t / (i+1)
        res += prev
    return res

P = get_markov_transition_prob(delta_t, A)

# option pricing params
sigma = max(vols) + (np.sqrt(1.5)-1) * np.mean(vols) # suggested by the paper
S0 = df['Close'].loc['2025-01-16'] # take a close price
risk_free_int = [.04, .04] # adjustable, or make them stochastic

risk_neutral_prob = { i: [-1]*3 for i in range(P.shape[0]) } # [up, mid, down]
for i in range(P.shape[0]):
    risk_neutral_prob[i][1] = 1 - (vols[i]/sigma) ** 2
    risk_neutral_prob[i][0] = (np.exp(risk_free_int[i]*delta_t)- np.exp(-sigma * np.sqrt(delta_t))- \
                               (1-(vols[i]/sigma)**2)*(1-np.exp(-sigma * np.sqrt(delta_t))))/(np.exp(sigma * np.sqrt(delta_t)) - np.exp(-sigma * np.sqrt(delta_t)))
    risk_neutral_prob[i][2] = (np.exp(sigma * np.sqrt(delta_t)) - np.exp(risk_free_int[i]*delta_t)- \
                               (1-(vols[i]/sigma)**2)*(np.exp(sigma * np.sqrt(delta_t))-1))/(np.exp(sigma * np.sqrt(delta_t)) - np.exp(-sigma * np.sqrt(delta_t)))


def call_value(time, node, state, K, american=False):
    """
    V(t,n,j) be the value of the derivative at the nth node at time step t under the jth regime state.
    time starts from 0
    node index starts from 1 (from bottom)
    """
    if state >= P.shape[0]:
        raise Exception('Invalid state')

    assert node <= 2 * time + 1 
    
    @cache
    def V(t, n, i):
        if t == N:
            return max(0, S0*np.exp((n - 1 - N)*sigma*np.sqrt(delta_t))-K)
        res = 0
        for j in range(P.shape[0]): # regimes
            res += P[i][j] * (
                risk_neutral_prob[i][0] * V(t+1, n+2, j) +
                risk_neutral_prob[i][1] * V(t+1, n+1, j) +
                risk_neutral_prob[i][2] * V(t+1, n, j)
            )
        # number of node = 2n+1
        res *= np.exp(-risk_free_int[i]*delta_t)
        if american:
            intrinsic = max(0, S0*np.exp((n - 1 - t) * sigma * np.sqrt(delta_t)) - K)
            res = max(res, intrinsic)
        
        return res
    
    return V(time, node, state)
    

def put_value(time, node, state, K, american=False):
    if state >= P.shape[0]:
        raise Exception('Invalid state')
    
    assert node <= 2 * time + 1 
    
    @cache
    def V(t, n, i):
        if t == N:
            return max(0, K - S0*np.exp((n - 1 - N)*sigma*np.sqrt(delta_t)))
        
        res = 0
        for j in range(P.shape[0]):  # Iterate over regimes
            res += P[i][j] * (
                risk_neutral_prob[i][0] * V(t+1, n+2, j) +
                risk_neutral_prob[i][1] * V(t+1, n+1, j) +
                risk_neutral_prob[i][2] * V(t+1, n, j)
            )
        
        discounted_value = np.exp(-risk_free_int[i] * delta_t) * res
        
        if american:
            intrinsic_value = max(0, K - S0*np.exp((n - 1 - t) * sigma * np.sqrt(delta_t)))
            return max(discounted_value, intrinsic_value)
        
        return discounted_value
    
    return V(time, node, state)

In [ ]:
import pandas as pd

data = []

for state in [0, 1]:
    for is_american in [True, False]:
        option_type = "American" if is_american else "European"
        for strike in range(240, 281, 10):
            call_price = call_value(0, 1, state, strike, is_american)
            put_price = put_value(0, 1, state, strike, is_american)

            # Append a dictionary with results
            data.append({
                "State": state,
                "Type": option_type,
                "Strike": strike,
                "Call Price": call_price,
                "Put Price": put_price
            })

df = pd.DataFrame(data)

df.to_csv("trinomial_result.csv", index=False)


## American Monte Carlo

In [15]:
import numpy as np

class AmericanOptionsLSMC:
    """ Class for American options pricing using Longstaff-Schwartz (2001):
    "Valuing American Options by Simulation: A Simple Least-Squares Approach."
    Review of Financial Studies, Vol. 14, 113-147.
    S0 : float : initial stock/index level
    strike : float : strike price
    T : float : time to maturity (in year fractions)
    M : int : grid or granularity for time (in number of total points)
    r : float : constant risk-free short rate
    div :    float : dividend yield
    sigma :  float : volatility factor in diffusion term 
    
    Unitest(doctest): 
    >>> AmericanPUT = AmericanOptionsLSMC('put', 36., 40., 1., 50, 0.06, 0.06, 0.2, 10000  )
    >>> AmericanPUT.price
    4.4731177017712209
    """

    def __init__(self, option_type, S0, strike, T, M, r, div, sigma, simulations):
        try:
            self.option_type = option_type
            assert isinstance(option_type, str)
            self.S0 = float(S0)
            self.strike = float(strike)
            assert T > 0
            self.T = float(T)
            assert M > 0
            self.M = int(M)
            assert r >= 0
            self.r = float(r)
            assert div >= 0
            self.div = float(div)
            assert sigma > 0
            self.sigma = float(sigma)
            assert simulations > 0
            self.simulations = int(simulations)
        except ValueError:
            print('Error passing Options parameters')


        if option_type != 'call' and option_type != 'put':
            raise ValueError("Error: option type not valid. Enter 'call' or 'put'")
        if S0 < 0 or strike < 0 or T <= 0 or r < 0 or div < 0 or sigma < 0:
            raise ValueError('Error: Negative inputs not allowed')

        self.time_unit = self.T / float(self.M)
        self.discount = np.exp(-self.r * self.time_unit)

    @property
    def MCprice_matrix(self, seed = 123):
        """ Returns MC price matrix rows: time columns: price-path simulation """
        np.random.seed(seed)
        MCprice_matrix = np.zeros((self.M + 1, self.simulations), dtype=np.float64)
        MCprice_matrix[0,:] = self.S0
        for t in range(1, self.M + 1):
            brownian = np.random.standard_normal( self.simulations // 2)
            brownian = np.concatenate((brownian, -brownian))
            MCprice_matrix[t, :] = (MCprice_matrix[t - 1, :]
                                  * np.exp((self.r - self.sigma ** 2 / 2.) * self.time_unit
                                  + self.sigma * brownian * np.sqrt(self.time_unit)))
        return MCprice_matrix

    @property
    def MCpayoff(self):
        """Returns the inner-value of American Option"""
        if self.option_type == 'call':
            payoff = np.maximum(self.MCprice_matrix - self.strike,
                           np.zeros((self.M + 1, self.simulations),dtype=np.float64))
        else:
            payoff = np.maximum(self.strike - self.MCprice_matrix,
                            np.zeros((self.M + 1, self.simulations),
                            dtype=np.float64))
        return payoff

    @property
    def value_vector(self):
        value_matrix = np.zeros_like(self.MCpayoff)
        value_matrix[-1, :] = self.MCpayoff[-1, :]
        for t in range(self.M - 1, 0 , -1):
            regression = np.polyfit(self.MCprice_matrix[t, :], value_matrix[t + 1, :] * self.discount, 5)
            continuation_value = np.polyval(regression, self.MCprice_matrix[t, :])
            value_matrix[t, :] = np.where(self.MCpayoff[t, :] > continuation_value,
                                          self.MCpayoff[t, :],
                                          value_matrix[t + 1, :] * self.discount)

        return value_matrix[1,:] * self.discount


    @property
    def price(self): return np.sum(self.value_vector) / float(self.simulations)

In [16]:
# strike = 250

# mc_am_call = AmericanOptionsLSMC('call', S0, strike, 1, 1000, 0.04, 0, vols[1], 500)
# mc_am_put = AmericanOptionsLSMC('put', S0, strike, 1, 1000, 0.04, 0, vols[1], 500)

# print(mc_am_call.price, mc_am_put.price)

data = []

for state in [0, 1]:
    for strike in range(240, 281, 10):
        call_price = AmericanOptionsLSMC('call', S0, strike, 1, 1000, risk_free_int[state], 0, vols[state], 500).price
        put_price = AmericanOptionsLSMC('put', S0, strike, 1, 1000, risk_free_int[state], 0, vols[state], 500).price

        # Append a dictionary with results
        data.append({
            "State": state,
            "Type": 'American',
            "Strike": strike,
            "Call Price": call_price,
            "Put Price": put_price
        })

df = pd.DataFrame(data)

df.to_csv("LSMC_result.csv", index=False)

In [41]:
# Standard Monte Carlo for European Options
# https://www.kaggle.com/code/ypark4857/monte-carlo-simulation-of-european-option-pricing

import numpy as np

rng = np.random.default_rng(123)

def mc_call(S, K, T, r, sigma, N):
    
    # Initial Asset Pricie
    S_init = S
    
    # X follows a standard normal distribution
    X = rng.normal(0, 1, N)
    
    # The Distribution of asset prices at the Expiration of the Option
    ST = S_init * np.exp((r-0.5*sigma**2)*T + sigma*np.sqrt(T)*X)
    
    # The Discounted payoff of European call option at expiration
    fST = np.exp(-r*T) * np.maximum(ST-K, 0)
    
    # The option value by taking the expected discounted payoff
    price = np.mean(fST)
    variance = np.var(fST)

    return price, variance

def mc_put(S, K, T, r, sigma, N):
    S_init = S

    # X follows a standard normal distribution
    X = rng.normal(0, 1, N)
    
    # The Distribution of asset prices at the Expiration of the Option
    ST = S_init * np.exp((r-0.5*sigma**2)*T + sigma*np.sqrt(T)*X)
    
    # The Discounted payoff of European call option at expiration
    fST = np.exp(-r*T) * np.maximum(K-ST, 0)

    # The option value by taking the expected discounted payoff
    price = np.mean(fST)
    variance = np.var(fST)
    
    return price, variance

In [46]:
data = []

for state in [0, 1]:
    for strike in range(240, 281, 10):
        call_price, _ = mc_call(S0, strike, 1, risk_free_int[state], vols[state], 5000)
        put_price, _ = mc_put(S0, strike, 1, risk_free_int[state], vols[state], 5000)

        # Append a dictionary with results
        data.append({
            "State": state,
            "Type": 'European',
            "Strike": strike,
            "Call Price": call_price.item(),
            "Put Price": put_price.item()
        })

df = pd.DataFrame(data)
df.to_csv('LSMC_result.csv', mode='a', index=False, header=False)

In [48]:
price_lsmc = pd.read_csv('LSMC_result.csv')
price_tri = pd.read_csv('trinomial_result.csv')

price_merge = pd.merge(price_lsmc, price_tri, on=['State', 'Strike', 'Type'], suffixes=('_MC', '_Trinomial'))

price_merge.round(4).to_csv('combined_result.csv', index=False)

In [49]:
final = pd.read_csv('combined_result.csv')
final

,State,Type,Strike,Call Price_MC,Put Price_MC,Call Price_Trinomial,Put Price_Trinomial
0,0,American,240,11.9085,16.7102,13.0671,17.2859
1,0,American,250,8.4735,23.7876,9.3316,24.1829
2,0,American,260,5.0446,32.5285,6.4980,32.4314
3,0,American,270,3.3759,42.4580,4.4206,41.9907
4,0,American,280,1.3092,52.4299,2.9451,51.9907
5,1,American,240,16.8170,21.5973,16.0400,20.2528
6,1,American,250,13.1857,27.5173,12.2339,26.8159
7,1,American,260,10.0275,35.3251,9.1858,34.3604
8,1,American,270,6.8400,43.8674,6.7973,42.8101
9,1,American,280,4.9619,53.0056,4.9630,52.0756
